In [39]:
import tensorflow as tf
import pandas as pd
import numpy as np
from numpy import array
from numpy import asarray
from numpy import zeros
import re
import nltk
from nltk.corpus import stopwords

from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten
from keras.layers import GlobalMaxPooling1D
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer

#df = pd.Dataframe()
df_train = pd.read_csv('./measuring-customer-happiness/train_hp.csv', encoding='utf-8')
print(df_train.head(3))

   User_ID                                        Description  \
0  id10326  The room was kind of clean but had a VERY stro...   
1  id10327  I stayed at the Crown Plaza April -- - April -...   
2  id10328  I booked this hotel through Hotwire at the low...   

        Browser_Used Device_Used Is_Response  
0               Edge      Mobile   not happy  
1  Internet Explorer      Mobile   not happy  
2            Mozilla      Tablet   not happy  


In [40]:
# Data Preprocessing

def preprocess_text(sen):
    # Removing html tags
    sentence = remove_tags(sen)

    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sentence)

    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)

    # Removing multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)

    return sentence

TAG_RE = re.compile(r'<[^>]+>')

def remove_tags(text):
    return TAG_RE.sub('', text)

X = []
sentences = list(df_train['Description'])
for sen in sentences:
    X.append(preprocess_text(sen))

In [41]:
X[3]


'Stayed here with husband and sons on the way to an Alaska Cruise We all loved the hotel great experience Ask for room on the North tower facing north west for the best views We had high floor with stunning view of the needle the city and even the cruise ships We ordered room service for dinner so we could enjoy the perfect views Room service dinners were delicious too You are in perfect spot to walk everywhere so enjoy the city Almost forgot Heavenly beds were heavenly too '

In [42]:
# binary classification for happy and not_happy 

y = df_train['Is_Response']

y = np.array(list(map(lambda x: 1 if x=="happy" else 0, y)))

# 1) Simple Neural Network

In [43]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [44]:
# Prepare embedding layer
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)

X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [45]:
# Adding 1 because of reserved 0 index
vocab_size = len(tokenizer.word_index) + 1

maxlen = 100

X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)


In [46]:
embeddings_dictionary = dict()
glove_file = open('./glove.twitter.27B/glove.twitter.27B.100d.txt', encoding="utf8")

for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = asarray(records[1:], dtype='float32')
    embeddings_dictionary [word] = vector_dimensions
glove_file.close()

In [47]:
embedding_matrix = zeros((vocab_size, 100))
for word, index in tokenizer.word_index.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector


In [48]:
model = Sequential()
embedding_layer = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=maxlen , trainable=False)
model.add(embedding_layer)

model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

In [49]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 100, 100)          4154400   
_________________________________________________________________
flatten_2 (Flatten)          (None, 10000)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 10001     
Total params: 4,164,401
Trainable params: 10,001
Non-trainable params: 4,154,400
_________________________________________________________________
None


# Model training

In [50]:
history = model.fit(X_train, y_train, batch_size=128, epochs=6, verbose=1, validation_split=0.2)

Train on 24916 samples, validate on 6229 samples
Epoch 1/6
24916/24916 [==============================] - 1s 40us/step - loss: 0.4898 - acc: 0.7654 - val_loss: 0.4423 - val_acc: 0.7952
Epoch 2/6
24916/24916 [==============================] - 1s 23us/step - loss: 0.3885 - acc: 0.8239 - val_loss: 0.4562 - val_acc: 0.7882
Epoch 3/6
24916/24916 [==============================] - 1s 21us/step - loss: 0.3538 - acc: 0.8456 - val_loss: 0.4443 - val_acc: 0.7960
Epoch 4/6
24916/24916 [==============================] - 1s 24us/step - loss: 0.3290 - acc: 0.8575 - val_loss: 0.4748 - val_acc: 0.7871
Epoch 5/6
24916/24916 [==============================] - 1s 23us/step - loss: 0.3097 - acc: 0.8705 - val_loss: 0.4636 - val_acc: 0.7919
Epoch 6/6
24916/24916 [==============================] - 1s 22us/step - loss: 0.2969 - acc: 0.8761 - val_loss: 0.4746 - val_acc: 0.7937


# 2) Convolutional Neural Network

In [51]:
from keras.layers.convolutional import Conv1D
model = Sequential()

embedding_layer = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=maxlen , trainable=False)
model.add(embedding_layer)

model.add(Conv1D(128, 5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

In [52]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 100, 100)          4154400   
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 96, 128)           64128     
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 128)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 129       
Total params: 4,218,657
Trainable params: 64,257
Non-trainable params: 4,154,400
_________________________________________________________________
None


# Model Training & Evaluation 

In [53]:
history = model.fit(X_train, y_train, batch_size=128, epochs=6, verbose=1, validation_split=0.2)

score = model.evaluate(X_test, y_test, verbose=1)

Train on 24916 samples, validate on 6229 samples
Epoch 1/6
24916/24916 [==============================] - 10s 386us/step - loss: 0.4269 - acc: 0.8044 - val_loss: 0.3689 - val_acc: 0.8358
Epoch 2/6
24916/24916 [==============================] - 11s 442us/step - loss: 0.3236 - acc: 0.8630 - val_loss: 0.3441 - val_acc: 0.8491
Epoch 3/6
24916/24916 [==============================] - 11s 429us/step - loss: 0.2718 - acc: 0.8909 - val_loss: 0.3486 - val_acc: 0.8480
Epoch 4/6
24916/24916 [==============================] - 9s 378us/step - loss: 0.2319 - acc: 0.9115 - val_loss: 0.3296 - val_acc: 0.8592
Epoch 5/6
24916/24916 [==============================] - 9s 379us/step - loss: 0.1993 - acc: 0.9294 - val_loss: 0.3314 - val_acc: 0.8566
Epoch 6/6
7787/7787 [==============================] - 1s 185us/step


In [54]:
print("Test Score:", score[0])
print("Test Accuracy:", score[1])
print(score)

Test Score: 0.31890831587479024
Test Accuracy: 0.8627199178500095
[0.31890831587479024, 0.8627199178500095]


In [209]:
instance = X[57]

print(instance)

instance = tokenizer.texts_to_sequences(instance)


flat_list = []
for sublist in instance:
    for item in sublist:
        flat_list.append(item)

flat_list = [flat_list]

instance = pad_sequences(flat_list, padding='post', maxlen=maxlen)

model.predict(instance)

horrible hotel staff treated me with disgusting demeaning attitude THEY WILL NOT ONLY BE REPORTED TO CHOICE and m elite diamond but WILL NEVER EVER GO NEAR THIS HOTEL AGAIN left saying my stay was just ok would rather report them to their franchise than deal with them as they clearly don care about their guests Ironically their horrible way of manging this hotel and dealing with guests surely has got to hurt their business Treating your guests with respect makes them want to return and spend more duh cannot recommend this hotel based on my experience here Their rooms are disgusting and have bed bugs 


array([[0.5084815]], dtype=float32)

## Call Facebook API & create Dataframe (comment & timestamp)

In [203]:
def callFacebookApi():
    url_long = "https://graph.facebook.com/v5.0/me?fields=id%2Cname%2Cposts%7Bcomments%7D&access_token=EAAlcIv35CUUBANREEygggKozZBFTNubNFhwuDn0u3MDI1jHz4PYRGirDFxz7MoSMTz3AHu6ZCQdT0oBp0cM3a30fGzw1pfb27C6H5OGn5jxiV49TqK8yaBiZA6DKZAeR3r0yzGGFFMGZAGmf6lMDeh8elMMGFZA5z4pk8KfQ2uiDZAZB67gt1nfh"
    response = requests.get(url_long)
    json_data = json.loads(response.text)
    dictFb = {'comment': [], 'time': []}

    for i in range(len(json_data['posts']['data'])):
        for j in range(len(json_data['posts']['data'][i]['comments']['data'])):
            dictFb['comment'].append(json_data['posts']['data'][i]['comments']['data'][j]['message'])
            dictFb['time'].append(json_data['posts']['data'][i]['comments']['data'][j]['created_time'])
   
    df = pd.DataFrame(dictFb, columns=['comment', 'time'])
    return df

                                              comment  \
0                          second comment 456 negativ   
1                     third comment 789 positiv :) <3   
2                                 comment 123 positiv   
3             Beautiful destinations for good prices!   
4   Enjoy the spectacular beaches, our delicious f...   
5                    High quality and a great crew!!!   
6   I can't understand the other (bad) comments. I...   
7   Dreamy beaches, nice views, great restaurants....   
8   Great service and crew!!!!!! Thank you for the...   
9   I had a very comfortable stay on board. I slep...   
10  I will never come back!!! On board it is dirty...   
11          The staff seemed unpolite, not motivated!   
12               Expensive and disappointing holiday.   
13  There is really nothing good I can say about t...   
14  The doctor on board makes overpriced bills and...   
15                            You ruined my vacation!   
16                         I do

## Sentiment Analysis

In [214]:
def analyze():
    
    negcount = 0
    poscount = 0
    negseries = 0
    
    reportneg = False
    
    df_comments = callFacebookApi()
    #print(df_comments)
    # iterate over dataframe
    for el in df_comments['comment']:
        el = tokenizer.texts_to_sequences(el)
        flat_list = []
        for sublist in el:
            for item in sublist:
                flat_list.append(item)
        flat_list = [flat_list]
        el = pad_sequences(flat_list, padding='post', maxlen=maxlen)
        
        predictvalue = model.predict(el)
        
        if (predictvalue <= 0.5):
            negcount += 1
            negseries += 1 
            sentiment = 'negativ'
            df_comments.insert(2, "Sentiment", sentiment, True)
            
        elif (predictvalue > 0.5):
            poscount += 1
            negseries = 0
            sentiment = 'positiv'
            df_comments.insert(2, "Sentiment", sentiment, True)
            
        if(negseries == 5 and reportneg == False):
            reportneg = True
            
            
    return {'df_comments':df_comments,'negative': negcount, 'positive': poscount,'total': negcount+poscount, 'reportnegative':reportneg,
           'negativepercent': negcount/(negcount+poscount)*100, 'positivepercent': poscount/(negcount+poscount)*100}
analyze()



                                              comment  \
0                          second comment 456 negativ   
1                     third comment 789 positiv :) <3   
2                                 comment 123 positiv   
3             Beautiful destinations for good prices!   
4   Enjoy the spectacular beaches, our delicious f...   
5                    High quality and a great crew!!!   
6   I can't understand the other (bad) comments. I...   
7   Dreamy beaches, nice views, great restaurants....   
8   Great service and crew!!!!!! Thank you for the...   
9   I had a very comfortable stay on board. I slep...   
10  I will never come back!!! On board it is dirty...   
11          The staff seemed unpolite, not motivated!   
12               Expensive and disappointing holiday.   
13  There is really nothing good I can say about t...   
14  The doctor on board makes overpriced bills and...   
15                            You ruined my vacation!   
16                         I do

{'df_comments':                                               comment  \
 0                          second comment 456 negativ   
 1                     third comment 789 positiv :) <3   
 2                                 comment 123 positiv   
 3             Beautiful destinations for good prices!   
 4   Enjoy the spectacular beaches, our delicious f...   
 5                    High quality and a great crew!!!   
 6   I can't understand the other (bad) comments. I...   
 7   Dreamy beaches, nice views, great restaurants....   
 8   Great service and crew!!!!!! Thank you for the...   
 9   I had a very comfortable stay on board. I slep...   
 10  I will never come back!!! On board it is dirty...   
 11          The staff seemed unpolite, not motivated!   
 12               Expensive and disappointing holiday.   
 13  There is really nothing good I can say about t...   
 14  The doctor on board makes overpriced bills and...   
 15                            You ruined my vacation!   

## Check for 5 negative comments or 40% comments

## Sentiment Analysis

## Timer

In [95]:
import json
import requests
import pandas as pd
import datetime
from threading import Thread, Event
import time


# Event object used to send signals from one thread to another
stop_event = Event()

def do_actions():
    """
    Function that should timeout after 5 seconds. It simply prints a number and waits 1 second.
    :return:
    """
    i = 0
    while True:
        callFacebookApi()
        # 1 second
        #time.sleep(1)
        # 30 seconds
        time.sleep(30)
        # 1 day
        #time.sleep(3600) # 1 hour 
        # Here we make the check if the other thread sent a signal to stop execution.
        if stop_event.is_set():
            break


if __name__ == '__main__':
    # We create another Thread
    action_thread = Thread(target=do_actions)

    # Here we start the thread and we wait 5 seconds before the code continues to execute.
    action_thread.start()
    action_thread.join(timeout=90) # 1,5 minutes
    #action_thread.join(timeout=86400) # 1 day

    # We send a signal that the other thread should stop.
    stop_event.set()

    print("Hey there! I timed out! You can do things after me!") 
    
from threading import Thread, Event
import time
 
# Event object used to send signals from one thread to another
stop_event = Event()

def do_actions():
    """
    Function that should timeout after 5 seconds. It simply prints a number and waits 1 second.
    :return:
    """
    i = 0
    while True:
        callFacebookApi()
        # 1 second
        #time.sleep(1)
        # 30 seconds
        time.sleep(30)
        # 1 day
        #time.sleep(3600) # 1 hour 
        # Here we make the check if the other thread sent a signal to stop execution.
        if stop_event.is_set():
            break
 
 
if __name__ == '__main__':
    # We create another Thread
    action_thread = Thread(target=do_actions)
 
    # Here we start the thread and we wait 5 seconds before the code continues to execute.
    action_thread.start()
    action_thread.join(timeout=90) # 1,5 minutes
    #action_thread.join(timeout=86400) # 1 day
 
    # We send a signal that the other thread should stop.
    stop_event.set()
 
    print("Hey there! I timed out! You can do things after me!")

{'id': '120017582730394', 'name': 'Lachmann Cruises', 'posts': {'data': [{'comments': {'data': [{'created_time': '2019-12-06T18:59:43+0000', 'from': {'name': 'Lachmann Cruises', 'id': '120017582730394'}, 'message': 'second comment 456 negativ', 'id': '160456975353121_165097008222451'}, {'created_time': '2019-12-06T19:00:01+0000', 'from': {'name': 'Lachmann Cruises', 'id': '120017582730394'}, 'message': 'third comment 789 positiv :) <3', 'id': '160456975353121_165097384889080'}, {'created_time': '2019-12-06T16:10:22+0000', 'from': {'name': 'Lachmann Cruises', 'id': '120017582730394'}, 'message': 'comment 123 positiv', 'id': '160456975353121_165045801560905'}], 'paging': {'cursors': {'before': 'MwZDZD', 'after': 'MQZDZD'}}}, 'id': '120017582730394_160456975353121'}, {'comments': {'data': [{'created_time': '2019-12-13T14:53:33+0000', 'from': {'name': 'Lachmann Cruises', 'id': '120017582730394'}, 'message': 'Beautiful destinations for good prices!', 'id': '160454742020011_168173377914814'}

Exception in thread Thread-4:
Traceback (most recent call last):
  File "/anaconda3/lib/python3.7/threading.py", line 917, in _bootstrap_inner
    self.run()
  File "/anaconda3/lib/python3.7/threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-95-c2d3112d6aad>", line 42, in do_actions
    callFacebookApi()
  File "<ipython-input-95-c2d3112d6aad>", line 25, in callFacebookApi
    datetime.datetime.strptime(df['time'], "%Y-%m-%dT%H:%M:%S+0000")
UnboundLocalError: local variable 'df' referenced before assignment



{'id': '120017582730394', 'name': 'Lachmann Cruises', 'posts': {'data': [{'comments': {'data': [{'created_time': '2019-12-06T18:59:43+0000', 'from': {'name': 'Lachmann Cruises', 'id': '120017582730394'}, 'message': 'second comment 456 negativ', 'id': '160456975353121_165097008222451'}, {'created_time': '2019-12-06T19:00:01+0000', 'from': {'name': 'Lachmann Cruises', 'id': '120017582730394'}, 'message': 'third comment 789 positiv :) <3', 'id': '160456975353121_165097384889080'}, {'created_time': '2019-12-06T16:10:22+0000', 'from': {'name': 'Lachmann Cruises', 'id': '120017582730394'}, 'message': 'comment 123 positiv', 'id': '160456975353121_165045801560905'}], 'paging': {'cursors': {'before': 'MwZDZD', 'after': 'MQZDZD'}}}, 'id': '120017582730394_160456975353121'}, {'comments': {'data': [{'created_time': '2019-12-13T14:53:33+0000', 'from': {'name': 'Lachmann Cruises', 'id': '120017582730394'}, 'message': 'Beautiful destinations for good prices!', 'id': '160454742020011_168173377914814'}

Exception in thread Thread-5:
Traceback (most recent call last):
  File "/anaconda3/lib/python3.7/threading.py", line 917, in _bootstrap_inner
    self.run()
  File "/anaconda3/lib/python3.7/threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-95-c2d3112d6aad>", line 81, in do_actions
    callFacebookApi()
  File "<ipython-input-95-c2d3112d6aad>", line 25, in callFacebookApi
    datetime.datetime.strptime(df['time'], "%Y-%m-%dT%H:%M:%S+0000")
UnboundLocalError: local variable 'df' referenced before assignment



In [ ]:
instance = tokenizer.texts_to_sequences(instance)

flat_list = []
for sublist in instance:
    for item in sublist:
        flat_list.append(item)

flat_list = [flat_list]

instance = pad_sequences(flat_list, padding='post', maxlen=maxlen)

model.predict(instance)

## Generate PDF Report

In [ ]:
pip install 

In [230]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as dt
from fpdf import FPDF
from datetime import date

In [221]:
pip install fpdf
pip install

  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40722 sha256=e584ce5f992b1eaac0ea77abbc5ec3943299cdb35bb9b71ab18d09bd4ab666c0
  Stored in directory: /Users/jf/Library/Caches/pip/wheels/9a/e9/77/4554ff5c99bc3f487c8d69620d8c41d99d54e9c54ab20ef4c9
Successfully built fpdf
Note: you may need to restart the kernel to use updated packages.


In [231]:
class Report:
    fakecsv = None

    def __init__(self, fakecsv):
        self.fakecsv = fakecsv

    #Variablen
    def initialize(self):
        count_minus1_neg = 0
        count_minus7_neg = 0
        count_minus31_neg = 0
        count_minus365_neg = 0
        count_minus1_pos = 0
        count_minus7_pos = 0
        count_minus31_pos = 0
        count_minus365_pos = 0
        now = datetime.now()
        minus1 = datetime.today() - timedelta(days=1)
        minus7 = datetime.today() - timedelta(days=7)
        minus31 = datetime.today() - timedelta(days=31)
        minus365 = datetime.today() - timedelta(days=365)

        fakedata = np.loadtxt(self.fakecsv, delimiter=';')

        df=pd.DataFrame()
        df['id'] = fakedata[:, 0]
        df['date'] = fakedata[:, 1]
        df['month'] = fakedata[:, 2]
        df['kw'] = fakedata[:, 3]
        df['weekday'] = fakedata[:, 4]
        df['day'] = fakedata[:, 7]
        df['hour'] = fakedata[:, 8]
        df['minute'] = fakedata[:, 9]
        df['weekhour'] = fakedata[:, 10]
        df['sentiment'] = fakedata[:, 11]
        df['real_date'] = pd.TimedeltaIndex(df['date'], unit='d') + dt.datetime(1900,1,1)

        array_minus1_neg = df[(df.real_date > minus1) & (df.sentiment == 2)].count()
        array_minus7_neg = df[(df.real_date > minus7) & (df.sentiment == 2)].count()
        array_minus31_neg = df[(df.real_date > minus31) & (df.sentiment == 2)].count()
        array_minus365_neg = df[(df.real_date > minus365) & (df.sentiment == 2)].count()

        count_minus1_neg = array_minus1_neg[0]
        count_minus7_neg = array_minus7_neg[0]
        count_minus31_neg = array_minus31_neg[0]
        count_minus365_neg = array_minus365_neg[0]

        array_minus1_pos = df[(df.real_date > minus1) & (df.sentiment == 1)].count()
        array_minus7_pos = df[(df.real_date > minus7) & (df.sentiment == 1)].count()
        array_minus31_pos = df[(df.real_date > minus31) & (df.sentiment == 1)].count()
        array_minus365_pos = df[(df.real_date > minus365) & (df.sentiment == 1)].count()

        count_minus1_pos = array_minus1_pos[0]
        count_minus7_pos = array_minus7_pos[0]
        count_minus31_pos = array_minus31_pos[0]
        count_minus365_pos = array_minus365_pos[0]

        sns.set(style="white", rc={"axes.facecolor": (0, 0, 0, 0)})

        # Create the data
        rs = np.random.RandomState(1979)
        x = rs.randn(500)
        g = np.tile(list("ABCDEFGHIJ"), 50)
        df = pd.DataFrame(dict(x=x, g=g))
        m = df.g.map(ord)
        df["x"] += m

        # Initialize the FacetGrid object
        pal = sns.cubehelix_palette(10, rot=-.25, light=.7)
        g = sns.FacetGrid(dfcsv, row="g", hue="g", aspect=15, height=.5, palette=pal)

        # Draw the densities in a few steps
        g.map(sns.kdeplot, "x", clip_on=False, shade=True, alpha=1, lw=1.5, bw=.2)
        g.map(sns.kdeplot, "x", clip_on=False, color="w", lw=2, bw=.2)
        g.map(plt.axhline, y=0, lw=2, clip_on=False)


        # Define and use a simple function to label the plot in axes coordinates
        def label(x, color, label):
            ax = plt.gca()
            ax.text(0, .2, label, fontweight="bold", color=color,
                    ha="left", va="center", transform=ax.transAxes)


        g.map(label, "x")

        # Set the subplots to overlap
        g.fig.subplots_adjust(hspace=-.25)

        # Remove axes details that don't play well with overlap
        g.set_titles("")
        g.set(yticks=[])
        g.despine(bottom=True, left=True)

        g.savefig("WeekPlot.png")

        sns.set(style="whitegrid")

        #rs = np.random.RandomState(365)
        #values = rs.randn(365, 4).cumsum(axis=0)
        #dates = pd.date_range("1 1 2016", periods=365, freq="D")
        values = df[(df.real_date > minus1)].count()
        print(values)
        hour = df.hour
        print(hour)
        data = pd.DataFrame(values, hour, columns=["Positiv", "Negativ", "Gesant"])
        data = data.rolling(7).mean()

        dayplot = sns.lineplot(data=data, palette="tab10", linewidth=2.5)
        print(dayplot)
        dayplotfigure = dayplot.get_figure()
        dayplotfigure.savefig("DayPlot.png")

        today = date.today()
        strtoday = today.strftime("%d.%m.%Y")

        slash = '/'
        string_neg1 = str(count_minus1_neg)
        string_pos1 = str(count_minus1_pos)
        string1 = string_neg1 + slash + string_pos1
        string_neg7 = str(count_minus7_neg)
        string_pos7 = str(count_minus7_pos)
        string7 = string_neg7 + slash + string_pos7
        string_neg31 = str(count_minus31_neg)
        string_pos31 = str(count_minus31_pos)
        string31 = string_neg31 + slash + string_pos31
        string_neg365 = str(count_minus365_neg)
        string_pos365 = str(count_minus365_pos)
        string365 = string_neg365 + slash + string_pos365

        pdf = FPDF()
        pdf.add_page()
        pdf.set_xy(0, 0)
        pdf.image('
                  .png', x = 0, y = 0, w = 210, h = 0, type = '', link = '')
        pdf.set_font('arial', '', 14)
        pdf.set_xy(108, 111)
        pdf.image('DayPlot.png', x = None, y = None, w = 92, h = 0, type = '', link = '')
        pdf.set_xy(19, 198)
        pdf.image('WeekPlot.png', x = None, y = None, w = 81, h = 0, type = '', link = '')
        pdf.set_xy(110, 198)
        pdf.image('WeekPlot2.png', x = None, y = None, w = 81, h = 0, type = '', link = '')
        pdf.set_xy(21, 127)
        pdf.cell(40, 10, str(string1))
        pdf.set_xy(67, 127)
        pdf.cell(40, 10, str(string7))
        pdf.set_xy(21, 161)
        pdf.cell(40, 10, str(string31))
        pdf.set_xy(67, 161)
        pdf.cell(40, 10, str(string365))
        pdf.set_xy(177, 75)
        pdf.cell(40, 10, strtoday)
        pdf = pdf.output('HamburgAnalytica.pdf', 'F')

        return pdf





In [232]:
report = Report('./FakeData.csv')
report.initialize()

AttributeError: module 'datetime' has no attribute 'now'

# Send email

In [ ]:
import smtplib, ssl

port = 587  # For starttls
smtp_server = "smtp.gmail.com"
receiver_email = "burghard.lachmann@gmail.com"
sender_email = "HH.Analytica@gmail.com"
password = 'WduenPa!19'
message = """
Report von HH Analytica
"""

context = ssl.create_default_context()
with smtplib.SMTP(smtp_server, port) as server:
   # server.ehlo()  # Can be omitted
    server.starttls(context=context)
    #server.ehlo()  # Can be omitted
    server.login(sender_email, password)
    server.sendmail(sender_email, receiver_email, message)

## attach PDF Report

In [ ]:
from email.MIMEMultipart import MIMEMultipart
from email.MIMEText import MIMEText
from email.MIMEImage import MIMEImage
msg = MIMEMultipart()
msg.attach(MIMEText(file("/home/myuser/sample.pdf").read()))